In [1]:
!pip install fastapi uvicorn torch torchvision opencv-python python-multipart nest_asyncio

In [2]:
!pip install torch torchvision torchaudio

In [ ]:
import nest_asyncio
from fastapi import FastAPI, File, UploadFile
from fastapi.responses import JSONResponse
from ultralytics import YOLO
import cv2
import numpy as np
import uvicorn
from pydantic import BaseModel

from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware

app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=["http://localhost:3000"],
    allow_credentials=True,
    allow_methods=["GET", "POST", "PUT", "DELETE"],
    allow_headers=["*"],
    expose_headers=["*"],
)

# Allow nested asyncio
nest_asyncio.apply()

# Load your YOLOv8 model
model = YOLO('best(1).pt')  # Adjust the path to your model

# Simple database of appliance wattages
appliance_wattages = {
    "fan": 53,
    "fridge": 212,
    "air conditioner": 3450,
    "washing machine": 500,
    "tubelight": 20,
    "oven": 2500,
    
    # Add more appliances as needed
}

class ApplianceInfo(BaseModel):
    type: str
    wattage: int

@app.post("/predict/")
async def predict(file: UploadFile = File(...)):
    try:
        # Read the image file
        contents = await file.read()
        nparr = np.frombuffer(contents, np.uint8)
        img = cv2.imdecode(nparr, cv2.IMREAD_COLOR)

        # Perform inference
        results = model(img)

        # Debug: Print the results
        print("Results:", results)

        # Process results
        max_confidence = 0
        best_class_name = None

        for result in results:
            # Check if boxes are present
            if result.boxes is not None:
                boxes = result.boxes.xyxy.numpy()  # Get bounding boxes
                confidences = result.boxes.conf.numpy()  # Get confidences
                classes = result.boxes.cls.numpy()  # Get class indices

                for conf, cls in zip(confidences, classes):
                    if conf > max_confidence:  # Update if this confidence is higher
                        max_confidence = conf
                        best_class_name = model.names[int(cls)]

        # Return the best class name and its wattage
        if best_class_name is not None:
            # Normalize the class name to lower case and remove extra spaces
            normalized_class_name = best_class_name.lower().strip()
            wattage = appliance_wattages.get(normalized_class_name, 0)
            return JSONResponse(content={"type": best_class_name, "wattage": wattage})

        return JSONResponse(content={"message": "No objects detected."})

    except Exception as e:
        return JSONResponse(content={"error": str(e)}, status_code=500)

@app.get("/appliance_info/{appliance_type}")
async def get_appliance_info(appliance_type: str):
    # Normalize the input to lower case and strip any extra spaces
    normalized_appliance_type = appliance_type.lower().strip()
    wattage = appliance_wattages.get(normalized_appliance_type, 0)
    return ApplianceInfo(type=appliance_type, wattage=wattage)

if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=7000)


WARNING  Python>=3.10 is required, but Python==3.9.13 is currently installed 


INFO:     Started server process [11576]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:7000 (Press CTRL+C to quit)
